In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thesis-version19/input_ann_ac_opf_val_ds69.csv
/kaggle/input/thesis-version19/output_ann_ac_opf_train_ds69.csv
/kaggle/input/thesis-version19/input_ann_ac_opf_train_ds69.csv
/kaggle/input/thesis-version19/output_ann_ac_opf_val_ds69.csv
/kaggle/input/thesis-version19/input_ann_ac_opf_test_ds69.csv
/kaggle/input/thesis-version19/output_ann_ac_opf_test_ds69.csv


# Version 19

### Import libraries

In [2]:
import pandas as pd
import tensorflow as tf
import time
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.python.client import device_lib
from sklearn.metrics import mean_squared_error

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Check available GPUs

In [3]:
devices = device_lib.list_local_devices()

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

 /device:CPU:0 || Unnamed device || CPU || 256.0 MiB
 /device:GPU:0 ||  Tesla P100-PCIE-16GB || GPU || 15.0 GiB


In [4]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15230107979839544094
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16120545280
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17661165542402061578
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]


In [5]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU detected. Please check your GPU configuration.")

Default GPU Device: /device:GPU:0


### Import the raw training, validation, and test sets

##### Training set

In [7]:
input_train = pd.read_csv('/kaggle/input/thesis-version19/input_ann_ac_opf_train_ds69.csv')
input_train.tail()

,snapshot,pdnet_1,pdnet_2,pdnet_3,pdnet_4,pdnet_5,pdnet_6,pdnet_7,pdnet_8,pdnet_9,...,offer_5_first_mg5,offer_32_first_mg6,offer_42_first_mg7,offer_63_second_mg1,offer_56_second_mg2,offer_23_second_mg3,offer_15_second_mg4,offer_5_second_mg5,offer_32_second_mg6,offer_42_second_mg7
56059,-28028,0.0,0.017353,0.0,0.015664,0.0,0.002532,0.031043,0.070855,0.027149,...,0.164988,0.334097,0.345680,0.132709,-1.0,0.869609,0.132653,0.170589,0.338330,0.345678
56060,-28029,0.0,0.017212,0.0,0.015828,0.0,0.002516,0.031013,0.071400,0.027300,...,0.165325,0.335413,0.343908,0.133272,-1.0,0.872408,0.132756,0.170077,0.338344,0.346371
56061,-28030,0.0,0.017210,0.0,0.016073,0.0,0.002513,0.031044,0.071231,0.027205,...,0.163669,0.334753,0.342827,0.132761,-1.0,0.863011,0.132193,0.169738,0.335994,0.344294
56062,-28031,0.0,0.017207,0.0,0.016094,0.0,0.002504,0.030764,0.071492,0.027493,...,0.164486,0.335395,0.342090,0.132924,-1.0,0.864054,0.132163,0.170254,0.336692,0.347065
56063,-28032,0.0,0.016980,0.0,0.016040,0.0,0.002474,0.030536,0.070742,0.027208,...,0.163211,0.335895,0.343642,0.132487,-1.0,0.864423,0.132753,0.171007,0.335295,0.345700


In [8]:
output_train = pd.read_csv('/kaggle/input/thesis-version19/output_ann_ac_opf_train_ds69.csv')
output_train.tail()

,snapshot,pg_1,pg_7,pg_27,pg_35,pg_37,pg_46,pg_50,pg_65,pg_67,...,lmp_61,lmp_62,lmp_63,lmp_64,lmp_65,lmp_66,lmp_67,lmp_68,lmp_69,objective_val
56059,-28028,0.994000,0.554713,0.573395,0.228941,0.318071,0.272776,0.799919,0.992901,0.497659,...,0.589744,0.579735,0.568516,0.514550,0.412563,0.329932,0.330127,0.320422,0.318710,0.693530
56060,-28029,1.000000,0.550721,0.565478,0.227220,0.310607,0.271230,0.795840,0.996801,0.494813,...,0.588029,0.580460,0.568065,0.515234,0.410291,0.328758,0.328889,0.319618,0.319450,0.687237
56061,-28030,0.992801,0.546995,0.561701,0.224834,0.304573,0.266172,0.796799,0.999000,0.492952,...,0.587849,0.577296,0.566098,0.513894,0.408936,0.330831,0.329703,0.320106,0.320291,0.687951
56062,-28031,0.996800,0.538421,0.555908,0.221965,0.297356,0.264111,0.796799,0.996000,0.485423,...,0.583769,0.576723,0.567162,0.513535,0.410306,0.328957,0.329453,0.318595,0.319070,0.680648
56063,-28032,0.996000,0.520886,0.536636,0.213608,0.275090,0.253817,0.794373,0.995900,0.474570,...,0.577606,0.570599,0.562317,0.506532,0.407334,0.329600,0.328843,0.320312,0.317587,0.667081


In [9]:
input_train.shape

(56064, 174)

In [10]:
output_train.shape

(56064, 241)

##### Validation set

In [11]:
input_val = pd.read_csv('/kaggle/input/thesis-version19/input_ann_ac_opf_val_ds69.csv')
input_val.tail()

,snapshot,pdnet_1,pdnet_2,pdnet_3,pdnet_4,pdnet_5,pdnet_6,pdnet_7,pdnet_8,pdnet_9,...,offer_5_first_mg5,offer_32_first_mg6,offer_42_first_mg7,offer_63_second_mg1,offer_56_second_mg2,offer_23_second_mg3,offer_15_second_mg4,offer_5_second_mg5,offer_32_second_mg6,offer_42_second_mg7
3451,35036,0.0,0.010589,0.0,0.009837,0.0,0.001369,0.017602,0.042484,0.015901,...,0.123036,0.331073,0.338520,0.133011,0.132840,0.874240,0.133186,0.170751,0.337704,0.346424
3452,35037,0.0,0.010261,0.0,0.009542,0.0,0.001331,0.017129,0.041228,0.015508,...,0.122680,0.330968,0.338323,0.133303,0.133345,0.869481,0.133149,0.170940,0.337738,0.346418
3453,35038,0.0,0.009934,0.0,0.009247,0.0,0.001293,0.016656,0.039973,0.015114,...,0.122323,0.330817,0.338126,0.133049,0.133276,0.864721,0.133254,0.170606,0.337720,0.346411
3454,35039,0.0,0.009606,0.0,0.008951,0.0,0.001255,0.016183,0.038717,0.014720,...,0.121966,0.330666,0.169977,0.133380,0.133222,0.872945,0.133434,0.170788,0.337706,0.346405
3455,35040,0.0,0.009279,0.0,0.008656,0.0,0.001218,0.015710,0.037462,0.014327,...,0.121610,0.330516,0.169173,0.133086,0.133168,0.868187,0.133193,0.170996,0.337740,0.346448


In [12]:
output_val = pd.read_csv('/kaggle/input/thesis-version19/output_ann_ac_opf_val_ds69.csv')
output_val.tail()

,snapshot,pg_1,pg_7,pg_27,pg_35,pg_37,pg_46,pg_50,pg_65,pg_67,...,lmp_61,lmp_62,lmp_63,lmp_64,lmp_65,lmp_66,lmp_67,lmp_68,lmp_69,objective_val
3451,35036,0.582987,4.879572e-03,0.032038,2.729038e-03,1.242842e-06,2.782073e-03,0.246810,0.541573,0.063478,...,0.375819,0.370041,0.363608,0.343127,0.298948,0.310244,0.310231,0.303012,0.302985,0.149147
3452,35037,0.540879,1.214469e-03,0.027552,1.431654e-03,1.281811e-06,1.267100e-03,0.237088,0.490539,0.055713,...,0.372020,0.366359,0.360122,0.340856,0.298821,0.309865,0.309854,0.302836,0.302811,0.133632
3453,35038,0.498039,2.526925e-04,0.023059,7.537647e-04,1.346520e-06,5.968596e-04,0.227332,0.441220,0.047947,...,0.368418,0.362874,0.356836,0.338704,0.298698,0.309487,0.309477,0.302660,0.302638,0.118302
3454,35039,0.185318,1.790486e-07,0.002701,2.904603e-08,7.164019e-08,2.690151e-08,0.205071,0.370174,0.023811,...,0.363882,0.358488,0.352696,0.336024,0.298520,0.308309,0.308307,0.301945,0.301926,0.102338
3455,35040,0.131991,3.647225e-06,0.002192,5.975032e-07,1.498945e-06,5.523875e-07,0.194829,0.320321,0.015100,...,0.360253,0.354978,0.349386,0.333858,0.298395,0.307884,0.307884,0.301649,0.301631,0.086973


##### Test set (baseline/normal loading)

In [13]:
input_test = pd.read_csv('/kaggle/input/thesis-version19/input_ann_ac_opf_test_ds69.csv')
input_test.tail()

,snapshot,pdnet_1,pdnet_2,pdnet_3,pdnet_4,pdnet_5,pdnet_6,pdnet_7,pdnet_8,pdnet_9,...,offer_5_first_mg5,offer_32_first_mg6,offer_42_first_mg7,offer_63_second_mg1,offer_56_second_mg2,offer_23_second_mg3,offer_15_second_mg4,offer_5_second_mg5,offer_32_second_mg6,offer_42_second_mg7
3547,33596,0.0,0.010782,0.0,0.010164,0.0,0.001444,0.018410,0.043994,0.016691,...,0.126287,0.332858,0.340263,0.132824,0.133051,0.870696,0.133141,0.170498,0.337721,0.346405
3548,33597,0.0,0.010394,0.0,0.009822,0.0,0.001399,0.017847,0.042528,0.016217,...,0.125603,0.332683,0.340006,0.132828,0.133206,0.863781,0.133174,0.170514,0.337728,0.346386
3549,33598,0.0,0.010006,0.0,0.009480,0.0,0.001355,0.017284,0.041062,0.015744,...,0.125006,0.332509,0.339797,0.132832,0.132820,0.869852,0.133208,0.170530,0.337735,0.346415
3550,33599,0.0,0.009617,0.0,0.009138,0.0,0.001311,0.016721,0.039596,0.015270,...,0.124602,0.332334,0.339540,0.132835,0.132975,0.875945,0.133243,0.170546,0.337758,0.346396
3551,33600,0.0,0.009229,0.0,0.008796,0.0,0.001266,0.016158,0.038129,0.014796,...,0.124066,0.332160,0.339283,0.132839,0.133131,0.869009,0.133280,0.170561,0.337701,0.346425


In [14]:
output_test = pd.read_csv('/kaggle/input/thesis-version19/output_ann_ac_opf_test_ds69.csv')
output_test.tail()

,snapshot,pg_1,pg_7,pg_27,pg_35,pg_37,pg_46,pg_50,pg_65,pg_67,...,lmp_61,lmp_62,lmp_63,lmp_64,lmp_65,lmp_66,lmp_67,lmp_68,lmp_69,objective_val
3547,33596,0.906911,0.046207,0.065495,0.049319,2.961395e-06,0.074934,0.272532,0.985377,0.104079,...,0.402015,0.396287,0.388041,0.358325,0.300080,0.312225,0.312200,0.304316,0.304273,0.274677
3548,33597,0.862616,0.036873,0.060289,0.046504,4.436169e-06,0.071104,0.260998,0.925433,0.095550,...,0.397656,0.392073,0.384055,0.355709,0.299908,0.311809,0.311786,0.304112,0.304072,0.256042
3549,33598,0.821008,0.027964,0.055193,0.043750,4.506900e-07,0.067367,0.249581,0.865652,0.087247,...,0.393363,0.387923,0.380131,0.353138,0.299759,0.311404,0.311383,0.303914,0.303876,0.237423
3550,33599,0.776794,0.018633,0.050050,0.040942,4.433451e-07,0.063545,0.238048,0.805478,0.078748,...,0.389026,0.383731,0.376167,0.350543,0.299608,0.310990,0.310971,0.303713,0.303677,0.218795
3551,33600,0.734302,0.009564,0.045054,0.038165,4.410372e-07,0.059747,0.226599,0.746854,0.070288,...,0.384699,0.379550,0.372213,0.347956,0.299462,0.310577,0.310561,0.303516,0.303483,0.200180


### Remove the snapshot column

##### Training set

In [15]:
x_train = input_train.loc[:, input_train.columns != 'snapshot']
x_train.head()

,pdnet_1,pdnet_2,pdnet_3,pdnet_4,pdnet_5,pdnet_6,pdnet_7,pdnet_8,pdnet_9,pdnet_10,...,offer_5_first_mg5,offer_32_first_mg6,offer_42_first_mg7,offer_63_second_mg1,offer_56_second_mg2,offer_23_second_mg3,offer_15_second_mg4,offer_5_second_mg5,offer_32_second_mg6,offer_42_second_mg7
0,0.0,0.009392,0.0,0.008478,0.0,0.001234,0.015761,0.037276,0.014020,0.013820,...,0.121376,0.330499,0.168336,0.133179,0.132931,0.868444,0.133103,0.170598,0.337720,0.346436
1,0.0,0.009236,0.0,0.008323,0.0,0.001214,0.015505,0.036629,0.013787,0.013600,...,0.121202,0.330424,0.167880,0.132950,0.133033,0.874698,0.133147,0.170977,0.337739,0.346446
2,0.0,0.009080,0.0,0.008169,0.0,0.001194,0.015250,0.035983,0.013553,0.013381,...,0.121029,0.330349,0.167170,0.133264,0.133134,0.867962,0.133192,0.170816,0.337763,0.346408
3,0.0,0.008925,0.0,0.008014,0.0,0.001174,0.014994,0.035336,0.013319,0.013162,...,0.120720,0.330273,0.166460,0.133035,0.133237,0.874214,0.133237,0.170671,0.337722,0.346419
4,0.0,0.008769,0.0,0.007860,0.0,0.001154,0.014739,0.034689,0.013085,0.012943,...,0.120547,0.330198,0.166004,0.132808,0.133352,0.867481,0.133285,0.170526,0.337740,0.346430


In [16]:
y_train = output_train.loc[:, output_train.columns != 'snapshot']
y_train.head()

,pg_1,pg_7,pg_27,pg_35,pg_37,pg_46,pg_50,pg_65,pg_67,pg_69,...,lmp_61,lmp_62,lmp_63,lmp_64,lmp_65,lmp_66,lmp_67,lmp_68,lmp_69,objective_val
0,0.184504,2.138566e-07,0.003812,3.789181e-08,8.684769e-08,3.513894e-08,0.202780,0.393835,0.023450,0.016585,...,0.365599,0.360550,0.354529,0.337006,0.298579,0.308292,0.308289,0.302013,0.301993,0.106169
1,0.153514,1.401898e-07,0.001552,2.645772e-08,6.154343e-08,2.453935e-08,0.197335,0.363138,0.018473,0.011848,...,0.363457,0.358483,0.352585,0.335738,0.298502,0.308049,0.308048,0.301884,0.301865,0.097377
2,0.122210,6.167888e-06,0.003291,7.950533e-07,1.865459e-06,7.339808e-07,0.191875,0.331345,0.013044,0.006972,...,0.361159,0.356258,0.350477,0.334372,0.298423,0.307776,0.307776,0.301631,0.301613,0.088581
3,0.094870,9.371905e-07,0.000804,1.893727e-07,4.483848e-07,1.754731e-07,0.186602,0.302417,0.008204,0.002746,...,0.359029,0.354204,0.348543,0.333113,0.298351,0.307547,0.307548,0.301520,0.301503,0.079795
4,0.060785,3.059371e-06,0.000708,5.744026e-07,1.382662e-06,5.318577e-07,0.181013,0.271002,0.003594,0.001444,...,0.356875,0.352127,0.346588,0.331838,0.298272,0.307174,0.307177,0.301153,0.301137,0.071004


##### Validation set

In [17]:
x_val = input_val.loc[:, input_val.columns != 'snapshot']
x_val.head()

,pdnet_1,pdnet_2,pdnet_3,pdnet_4,pdnet_5,pdnet_6,pdnet_7,pdnet_8,pdnet_9,pdnet_10,...,offer_5_first_mg5,offer_32_first_mg6,offer_42_first_mg7,offer_63_second_mg1,offer_56_second_mg2,offer_23_second_mg3,offer_15_second_mg4,offer_5_second_mg5,offer_32_second_mg6,offer_42_second_mg7
0,0.0,0.009269,0.0,0.008364,0.0,0.001204,0.015449,0.036671,0.013784,0.013574,...,0.123013,0.331669,0.338814,0.132877,0.133197,0.865377,0.133353,0.170663,0.337710,0.346435
1,0.0,0.009128,0.0,0.008198,0.0,0.001183,0.015178,0.036001,0.013509,0.013336,...,0.122799,0.331584,0.338717,0.133094,0.133149,0.870176,0.133130,0.170874,0.337715,0.346430
2,0.0,0.008987,0.0,0.008032,0.0,0.001162,0.014906,0.035332,0.013235,0.013097,...,0.122585,0.331500,0.338572,0.133320,0.133100,0.874976,0.133213,0.170577,0.337722,0.346428
3,0.0,0.008846,0.0,0.007866,0.0,0.001141,0.014634,0.034662,0.012960,0.012859,...,0.122371,0.331415,0.338474,0.132986,0.133052,0.866789,0.133354,0.170788,0.337731,0.346426
4,0.0,0.008705,0.0,0.007700,0.0,0.001120,0.014362,0.033992,0.012685,0.012621,...,0.122157,0.331330,0.338329,0.133203,0.133003,0.871587,0.133112,0.171030,0.337737,0.346423


In [18]:
y_val = output_val.loc[:, output_val.columns != 'snapshot']
y_val.head()

,pg_1,pg_7,pg_27,pg_35,pg_37,pg_46,pg_50,pg_65,pg_67,pg_69,...,lmp_61,lmp_62,lmp_63,lmp_64,lmp_65,lmp_66,lmp_67,lmp_68,lmp_69,objective_val
0,0.645891,0.000223,0.034717,0.030015,4.086466e-07,0.048215,0.219220,0.608897,0.054357,0.057279,...,0.374421,0.369598,0.362884,0.342085,0.299117,0.309800,0.309788,0.303119,0.303092,0.164230
1,0.622111,0.000045,0.032059,0.028521,3.797762e-07,0.046181,0.214255,0.576292,0.049687,0.053551,...,0.371898,0.367164,0.360582,0.340600,0.299035,0.309572,0.309562,0.303017,0.302991,0.154969
2,0.597904,0.000376,0.029421,0.027087,3.332981e-06,0.044081,0.209272,0.543930,0.044835,0.049691,...,0.369374,0.364729,0.358280,0.339114,0.298954,0.309335,0.309326,0.302912,0.302886,0.145710
3,0.574603,0.000011,0.026727,0.025551,7.329421e-07,0.042120,0.204326,0.513106,0.040130,0.045941,...,0.367013,0.362460,0.356147,0.337731,0.298877,0.309106,0.309098,0.302809,0.302785,0.136453
4,0.550691,0.000007,0.024070,0.024055,8.672615e-07,0.040078,0.199355,0.480519,0.035444,0.042205,...,0.364494,0.360030,0.353849,0.336247,0.298796,0.308877,0.308871,0.302707,0.302684,0.127194


##### Test set (baseline/normal loading)

In [19]:
x_test = input_test.loc[:, input_test.columns != 'snapshot']
x_test.head()

,pdnet_1,pdnet_2,pdnet_3,pdnet_4,pdnet_5,pdnet_6,pdnet_7,pdnet_8,pdnet_9,pdnet_10,...,offer_5_first_mg5,offer_32_first_mg6,offer_42_first_mg7,offer_63_second_mg1,offer_56_second_mg2,offer_23_second_mg3,offer_15_second_mg4,offer_5_second_mg5,offer_32_second_mg6,offer_42_second_mg7
0,0.0,0.009288,0.0,0.008585,0.0,0.001198,0.015410,0.037141,0.013898,0.013510,...,0.121189,0.330390,0.167754,0.132815,0.132953,0.872450,0.133106,0.170921,0.337757,0.346422
1,0.0,0.009166,0.0,0.008424,0.0,0.001181,0.015185,0.036529,0.013659,0.013318,...,0.120914,0.330285,0.167420,0.133189,0.133114,0.869446,0.133130,0.170901,0.337736,0.346560
2,0.0,0.009045,0.0,0.008262,0.0,0.001163,0.014960,0.035917,0.013420,0.013125,...,0.120774,0.330225,0.166831,0.133013,0.133277,0.866440,0.133154,0.170882,0.337719,0.346443
3,0.0,0.008923,0.0,0.008101,0.0,0.001146,0.014736,0.035304,0.013181,0.012933,...,0.120498,0.330165,0.166241,0.132842,0.132895,0.863434,0.133177,0.170853,0.337752,0.346426
4,0.0,0.008802,0.0,0.007939,0.0,0.001128,0.014511,0.034692,0.012942,0.012741,...,0.120358,0.330060,0.165652,0.133216,0.133056,0.873411,0.133201,0.170833,0.337738,0.346411


In [20]:
y_test = output_test.loc[:, output_test.columns != 'snapshot']
y_test.head()

,pg_1,pg_7,pg_27,pg_35,pg_37,pg_46,pg_50,pg_65,pg_67,pg_69,...,lmp_61,lmp_62,lmp_63,lmp_64,lmp_65,lmp_66,lmp_67,lmp_68,lmp_69,objective_val
0,0.130768,7.084788e-07,0.001493,1.338271e-07,3.167359e-07,1.240069e-07,0.190359,0.335860,0.014531,0.006287,...,0.361600,0.356617,0.350847,0.334629,0.298434,0.307857,0.307857,0.301718,0.301701,0.088713
1,0.104716,9.938498e-07,0.000896,1.937713e-07,4.635586e-07,1.795200e-07,0.186390,0.308636,0.009919,0.003065,...,0.359551,0.354643,0.348974,0.333420,0.298366,0.307632,0.307633,0.301542,0.301525,0.081486
2,0.077306,3.578287e-06,0.000881,6.653205e-07,1.604832e-06,6.159531e-07,0.182355,0.281969,0.005544,0.001896,...,0.357655,0.352825,0.347263,0.332308,0.298299,0.307357,0.307359,0.301260,0.301244,0.074248
3,0.052379,2.328990e-06,0.000473,4.286457e-07,1.043731e-06,3.970570e-07,0.178438,0.255614,0.002497,0.000891,...,0.355805,0.351051,0.345591,0.331220,0.298234,0.307031,0.307034,0.300973,0.300957,0.067020
4,0.023189,2.767553e-07,0.000013,6.375364e-08,1.560775e-07,5.923743e-08,0.174318,0.226937,0.000145,0.000027,...,0.353744,0.349065,0.343707,0.330003,0.298162,0.306616,0.306620,0.300645,0.300630,0.059783


### Train ANN-MLP model

In [21]:
model = Sequential()
model.add(Dense(207, input_dim = 173, activation = 'relu'))
model.add(Dense(207, activation = 'relu'))
model.add(Dense(207, activation = 'relu'))
model.add(Dense(240))

model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mean_squared_error'])

In [22]:
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 50, restore_best_weights = True, verbose = 1)

In [23]:
start_time = time.time()
model.fit(x_train,
          y_train,
          validation_data = (x_val, y_val),
          batch_size = 48,
          epochs = 1000,
          callbacks = [early_stopping],
          verbose = 2)
end_time = time.time()

Epoch 1/1000
1168/1168 - 9s - loss: 0.0030 - mean_squared_error: 0.0030 - val_loss: 7.6874e-05 - val_mean_squared_error: 7.6874e-05 - 9s/epoch - 8ms/step
Epoch 2/1000
1168/1168 - 4s - loss: 1.0222e-04 - mean_squared_error: 1.0222e-04 - val_loss: 4.1444e-05 - val_mean_squared_error: 4.1444e-05 - 4s/epoch - 3ms/step
Epoch 3/1000
1168/1168 - 4s - loss: 7.5462e-05 - mean_squared_error: 7.5462e-05 - val_loss: 2.7914e-05 - val_mean_squared_error: 2.7914e-05 - 4s/epoch - 3ms/step
Epoch 4/1000
1168/1168 - 4s - loss: 4.9101e-05 - mean_squared_error: 4.9101e-05 - val_loss: 2.6130e-05 - val_mean_squared_error: 2.6130e-05 - 4s/epoch - 3ms/step
Epoch 5/1000
1168/1168 - 4s - loss: 4.7052e-05 - mean_squared_error: 4.7052e-05 - val_loss: 1.8081e-05 - val_mean_squared_error: 1.8081e-05 - 4s/epoch - 3ms/step
Epoch 6/1000
1168/1168 - 4s - loss: 3.7876e-05 - mean_squared_error: 3.7876e-05 - val_loss: 1.6757e-05 - val_mean_squared_error: 1.6757e-05 - 4s/epoch - 3ms/step
Epoch 7/1000
1168/1168 - 4s - loss: 

In [24]:
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time) 

Elapsed time:  1096.043930053711


### Save the model

In [25]:
model.save('model.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Perform prediction on the test set using the model

In [26]:
y_test_pred = model.predict(x_test)
y_test_pred = pd.DataFrame(data = y_test_pred)
y_test_pred.head()

111/111 [==============================] - 0s 2ms/step


,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
0,0.125334,0.002626,0.002776,0.000782,0.000548,0.001430,0.191578,0.332743,0.016223,0.007682,...,0.361153,0.356674,0.350538,0.335629,0.299325,0.307424,0.308312,0.301570,0.301463,0.088204
1,0.098821,0.002013,0.001490,0.000759,0.000529,0.001483,0.187257,0.304957,0.011919,0.004802,...,0.359268,0.354802,0.348761,0.334367,0.299278,0.307166,0.308061,0.301459,0.301315,0.080748
2,0.076170,0.001719,0.000185,0.000218,0.000397,0.000931,0.182505,0.278738,0.008048,0.001997,...,0.357581,0.353049,0.347043,0.333092,0.299152,0.306870,0.307778,0.301294,0.301208,0.073992
3,0.055009,0.001046,-0.001323,-0.000278,0.000354,0.000195,0.177679,0.253299,0.004384,-0.001253,...,0.355837,0.351230,0.345298,0.331895,0.298977,0.306553,0.307482,0.301066,0.301058,0.067445
4,0.031275,0.000508,-0.003078,-0.001317,0.000143,-0.000960,0.172085,0.224414,0.000742,-0.004590,...,0.353704,0.349014,0.343187,0.330308,0.298704,0.306237,0.307168,0.300798,0.300879,0.060013


### Evaluate the model on the test set

In [27]:
score_1 = model.evaluate(x_test, y_test)
print('Test loss: ', score_1[0])

111/111 [==============================] - 0s 3ms/step - loss: 1.0272e-06 - mean_squared_error: 1.0272e-06
Test loss:  1.0272146937495563e-06


In [28]:
score_2 = mean_squared_error(y_test, y_test_pred)
print('Test loss: ', score_2)

Test loss:  1.0272148815782116e-06


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt